In [1]:
import os
os.chdir('/home/damon/Github/mmlab/mmdetection/')
import os.path as osp
import sys
import numpy as np
import torch, torchvision
from torch import nn
from torch.utils import model_zoo
from torch.nn.parallel import DataParallel
from torch.utils.data import DataLoader
from torch.utils.data.dataset import ConcatDataset
# %load_ext autoreload
# %autoreload 2
from torchsummary import summary

In [2]:
import mmdet
import mmcv
from mmdet.models import build_detector
from mmcv.parallel import MMDataParallel
from mmdet.datasets import get_dataset
from mmdet.datasets import transforms

In [3]:
# Get cfg
import mmcv
config_file = 'configs/retinanet_x101_64x4d_fpn_1x.py'
cfg = mmcv.Config.fromfile(config_file)
cfg.model.pretrained = None

# Get model
from mmdet.models import build_detector
det = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Dataset

# Backbone

In [4]:
input_size =  (3, 800, 800)
backbone = det.backbone
bbox_head = det.bbox_head

In [5]:
backbone

ResNeXt(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), str

In [ ]:
x = torch.zeros([1,3,300,300])
fea_map = backbone(x)

In [ ]:
[v.shape for v in fea_map]

In [6]:
summary(backbone.cuda(), (3, 300, 300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           9,408
       BatchNorm2d-2         [-1, 64, 150, 150]             128
              ReLU-3         [-1, 64, 150, 150]               0
         MaxPool2d-4           [-1, 64, 75, 75]               0
            Conv2d-5          [-1, 256, 75, 75]          16,384
       BatchNorm2d-6          [-1, 256, 75, 75]             512
              ReLU-7          [-1, 256, 75, 75]               0
            Conv2d-8          [-1, 256, 75, 75]           9,216
       BatchNorm2d-9          [-1, 256, 75, 75]             512
             ReLU-10          [-1, 256, 75, 75]               0
           Conv2d-11          [-1, 256, 75, 75]          65,536
      BatchNorm2d-12          [-1, 256, 75, 75]             512
           Conv2d-13          [-1, 256, 75, 75]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
summary(backbone, input_size)

In [ ]:
bbox_head

In [ ]:
reg_out = bbox_head(fea_map)[0]
[v.shape for v in reg_out]

In [ ]:
cls_out = bbox_head(fea_map)[1]
[v.shape for v in cls_out]

# bboxx_head loss

In [ ]:
lossinput = bbox_head(fea_map)

# Transfroms

In [ ]:
from mmdet.datasets import transforms

In [ ]:
img = np.zeros([ 400, 400, 3])
transforms.Numpy2Tensor()(img)

In [ ]:
bboxes = np.random.rand(20, 4)

In [ ]:
from mmcv.image.transforms import geometry

In [ ]:
geometry.impad(img, (411, 411), pad_val=2).shape

In [ ]:
pad = np.empty((411,411))